In [ ]:
from blockchain import blockexplorer
from os.path import exists
import networkx as nx
import sys
import pickle
import csv
import time
import random

## If something happened and importing got interrupted, it will look for previous log files and import them and pick up where it left off.  If not, the hash of the block you want to start at is needed

In [ ]:
#latestBlockHash = blockexplorer.get_latest_block().hash
#command = sys.argv[1]
#value = sys.argv[2]
command = "hash"
value = "0000000000000000003202a770a5ba275e759b998378e4375827d67aa150f343"

# If there are year files we want to inport for a certain year.
if command.lower() == "file":

	graphFileName = "graph" + value + ".csv"
	if exists(graphFileName):
		csvfile = open(graphFileName, "a")
	else:
		csvfile = open(graphFileName, "w")

	logFileName = "log" + value + ".txt"
	if exists(logFileName):
		blockFile = open(logFileName, "a")
	else:
		blockFile = open(logFileName, "w")

	latestBlockFile = "lastBlock" + value + ".txt"
	if exists(latestBlockFile):
		with open(latestBlockFile, "r") as latestSampledBlock:
			latestSampledBlockHash = latestSampledBlock.readline()
			currentBlock = blockexplorer.get_block(latestSampledBlockHash)
	else:
		sys.exit()

	stopTime = 1514684892055
	year = int(value)
	stopTime = stopTime - ((2018 - year) * 31536000000)
	stopTime = str(stopTime)
	stopTime = stopTime[:-3]
	stopTime = int(stopTime)


# If there is a block we want to start at
elif command.lower() == "hash":
	currentBlock = blockexplorer.get_block(value)
	blockFile = open("log.txt", "w")
	csvfile = open("graph.csv", "w")
	latestSampledBlockHash = open("lastBlock.txt", "w")
	stopTime = 0

### Will continue to import and sample the blockchain until it reaches the start of the year.

In [ ]:
running = True
numberOfBlocks = 0
writer = csv.writer(csvfile, delimiter = ',')

while running:

    print("Block Hash: ", currentBlock.hash)
	print("Block time: ", currentBlock.time)
	blockFile.write(currentBlock.hash)
	blockFile.write("\n")

	with open(latestBlockFile, "w") as latestSampledBlock:
		latestSampledBlock.write(currentBlock.hash)

	transactions = currentBlock.transactions

	if(transactions[0].time < stopTime):
		running = False

	transactionCounter = 0
	for transaction in transactions:
		blockFile.write("  ")
		blockFile.write(transaction.hash)
		blockFile.write("\n")

		inputs = transaction.inputs
		outputs = transaction.outputs

		isBlind = False
		# If the transactions has no input is is blind addition
		if not hasattr(inputs[0], 'address'):
			isBlind = True

		averageWithdraw = 0
		for out in outputs:
			averageWithdraw = averageWithdraw + out.value

		averageWithdraw = averageWithdraw / len(inputs) if not isBlind else averageWithdraw
		averageWithdraw = averageWithdraw / len(outputs) 

		# Decide whether or not we should keep goind.
		#if graphIndex < 0 or graphIndex > 0:
		#	running = False


		if isBlind:
			for output in outputs:
				if(output.address is not None):
					writer.writerow([output.address, output.address, int(averageWithdraw)])
		else:
			inputOdds = 1 / (2 * len(inputs))
			outputOdds = 1 / (2 * len(outputs)) 
			for input_ in inputs:
				if random.random() < inputOdds:
					for output in outputs:
						if random.random() < outputOdds:
							if hasattr(input,_ "address") and input_.address != output.address and input_.address is not None and output.address is not None:
								writer.writerow([input_.address, output.address])
								blockFile.write("    ")
								blockFile.write(input_.address)
								blockFile.write("->")
								blockFile.write(output.address)
								blockFile.write("\n")

		transactionCounter = transactionCounter + 1
	numberOfBlocks = numberOfBlocks + 1
	

	print("\tTransactions: ", transactionCounter)
	found = False
	while not found:
		try:
			currentBlock = blockexplorer.get_block(currentBlock.previous_block)
			found = True
		except:
			print("Sleeping...")
			time.sleep(10)